Let us take it as a revision and setup CICD Pipeline for the project.

Note: Please review the course material related to setting up CICD Pipeline before taking care of this.

* Setup as a Git Repo for `sales-app-rest`. Make sure gunicorn is added to the dependencies. Also, make sure to create `.gitignore` file and add the folders and files which are not supposed to be pushed into GitHub repo.
* Make sure to push the Git Repo to GitHub. Add required secrets in GitHub Repo.
* Stop Supervisor and ensure to remove existing configuration file for `sales-app` web application.
* Create a folder for `sales-app-rest` and take care of the following.
  * We need to have a `.env` file with database connectivity information. Make sure to create one with correct Database information. You can use the existing `env` file used for `sales-app` as reference.
  * Create a new supervisor configuration file by name `sales-app-rest.conf` for `sales-app-rest`. Move it to the appropriate location - `/etc/supervisor/conf.d`.

```text
[program:sales-app-rest]
directory=/home/itversity/sales-app-rest
command=/home/itversity/sales-app-rest/sar-venv/bin/gunicorn app:app -b localhost:5000
autostart=true
autorestart=true
stderr_logfile=/var/log/sales-app-rest/sales-app-rest.err.log
stdout_logfile=/var/log/sales-app-rest/sales-app-rest.out.log
```

* Add workflow in the GitHub Repo. I have used existing workflow as reference and made required changes.

```yaml
name: Sales App Rest

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Enable Passwordless Login
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
    - name: Create Folder and Deploy Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
      run: |
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p ~/sales-app-rest
        '
        scp -i /tmp/keys/private_key -r * ${APP_SSH_USER}@${APP_HOST}:~/sales-app-rest
    - name: Build and Run Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
      run: |
        ssh -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} "/bin/bash -c 'source ~/sales-app-rest/setup.sh'"
```

* Add shell script to build and run the application.

```shell
python3 -m venv /home/itversity/sales-app-rest/sar-venv
source /home/itversity/sales-app-rest/sar-venv/bin/activate
pip install -r /home/itversity/sales-app-rest/requirements.txt
sudo supervisorctl restart sales-app-rest
```

* Review GitHub Action Run. It might fail, make sure troubleshoot the issue.
* Validate Supervisor and Gunicorn. Review the logs to see if every thing is fine.
* Validate REST APIs using curl.

```shell
curl http://35.238.90.14/users
curl http://35.238.90.14/courses

# Optionally we can use 
# curl -X GET http://35.238.90.14/users
# Default is GET
curl -X GET http://35.238.90.14/users
```

In case if you would like to redo, here are the instructions to revert all the changes made for the CICD Pipeline:

1. On GCP VM, delete configuration file for `sales-app-rest` and restart supervisor.
2. On GCP VM, delete the logs folder and project folder.
3. On GitHub, go to settings and delete the repository.
4. On our local environment, delete .git folder, .github folder and also .gitignore files. You can also remove gunicorn from dependencies.